<a href="https://colab.research.google.com/github/shreyassathyamangalam/ai_projects/blob/main/Neo4j_Graphiti_KG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Dependencies

In [ ]:
!pip install -q -U graphiti-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.8/325.8 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 10.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.17.0 requires tenacity<9.0.0,>=8.0.0, but you have tenacity 9.1.2 which is incompatible.


## Connection setup

In [ ]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

neo4j_uri="neo4j+s://1d531fab.databases.neo4j.io"
neo4j_username="neo4j"
neo4j_password="mFkMTEbJV1fsfl3xpr42udGQl2jXCDTy_gxPAfYsVFc"
neo4j_database="neo4j"

In [ ]:
import asyncio
import json
from datetime import datetime, timezone

from graphiti_core import Graphiti
from graphiti_core.nodes import EpisodeType
from graphiti_core.search.search_config_recipes import NODE_HYBRID_SEARCH_RRF
from graphiti_core.utils.maintenance.graph_data_operations import clear_data

In [ ]:
import nest_asyncio, asyncio
nest_asyncio.apply()

## Initialize Graphiti with Neo4J Connection

In [ ]:
graphiti = Graphiti(neo4j_uri, neo4j_username, neo4j_password)

# Initialize the graph database with graphiti's indices. This only needs to be done once.
# Note: This will clear the database

await clear_data(graphiti.driver)
await graphiti.build_indices_and_constraints()

https://help.getzep.com/graphiti/core-concepts/adding-episodes

In [ ]:
episodes = [
    {
        "name": "About Me",
        "content": "Hi, I'm Shreyas. I'm learning how to build Knowledge Graphs",
        "type": EpisodeType.text,
        "description": "intro"
    },
    {
        "name": "About my Knowledge Graphs",
        "content": "Knowledge Graphs facinate me",
        "type": EpisodeType.text,
        "description": "overview"
    }
]

In [ ]:
async def add_data(episodes):
  """Add Episodes to the graph"""
  for episode in episodes:
    if episode["type"] == EpisodeType.json:
      episode["content"] = json.dumps(episode["content"])

    await graphiti.add_episode(
        name=episode["name"],
        episode_body=episode["content"],
        source=episode["type"],
        source_description=episode["description"],
        reference_time=datetime.now(timezone.utc)
        )
    print(f"Added episode {episode['name']}")

In [ ]:
await add_data(episodes)

Added episode About Me
Added episode About my Knowledge Graphs


In [ ]:
episodes = [
    {
        "name": "About AI Projects",
        "content": "I am behind a few AI Projects",
        "type": EpisodeType.text,
        "description": "overview",
    },
    {
        "name": "AI Projects",
        "content": {
            'product': 'Shreyas Agent',
            'company': 'Shreyas Company',
            'website': 'lovable portfolio'
        },
        "type": EpisodeType.json,
        "description": "project metadata",
    }
]

In [ ]:
await add_data(episodes)

Added episode About AI Projects
Added episode AI Projects


In [ ]:
# Perform a Hybrid Search combining semantic similarity and BM25 retrieval

query = "What products does AI Projects have?"
results = await graphiti.search(query=query, num_results=1)

In [ ]:
def print_results(results):
  """Print the results of a search query."""
  print('\nSearch Results:')
  for result in results:
    print(f"UUID: {result.uuid}")
    print(f"Fact: {result.fact}")
    if hasattr(result, 'valid_at') and result.valid_at:
      print(f"Valid from: {result.valid_at}")
    if hasattr(result, 'invalid_at') and result.invalid_at:
      print(f"Valid until: {result.invalid_at}")
    print('---')

In [ ]:
print_results(results)


Search Results:
UUID: 7b66b2a3-cb07-43c6-994b-450dd660aab8
Fact: I is behind a few AI Projects
Valid from: 2025-11-06 17:42:21.220805+00:00
Valid until: 2025-11-06 17:42:43.085685+00:00
---
